In [218]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [219]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
# options.add_argument("--disable-extensions")
# options.add_argument("disable-gpu")



In [220]:
# 데이터 불러오기
print(os.getcwd())
camping = pd.read_csv('../file/camping.csv')

/Users/yongjae/Dropbox/layerbylayer/frint


In [221]:
camping.head(3)

,name,name_modify,category,y_coord,x_coord,address_str,address_jibun,tel,num_site,area,...,conv,safety,etc_bldg,time,price,tel_admin,admin,collect_date,admin_code,admin_name
0,한탄강소나무캠핑,한탄강소나무캠핑,야영장,38.015842,127.077063,경기도 연천군 전곡읍 선사로286번길 37,경기도 연천군 전곡읍 전곡리 56-23,031-833-0000,105,73164.0,...,전기시설+상하수도시설+샤워실,소화기+대피공간,글램핑12동,NaN,NaN,031-839-2147,경기도 연천군청,2019-09-24,4140000,경기도 연천군
1,한탄강관광지오토캠핑장,한탄강관광지오토캠핑장,자동차야영장,38.008876,127.058860,경기도 연천군 전곡읍 선사로 76,경기도 연천군 전곡읍 전곡리 696-3,031-833-0030,1,3716.0,...,전기시설+상하수도시설+샤워실,소화기+대피공간,카라반55동,NaN,NaN,031-839-2147,경기도 연천군청,2019-09-24,4140000,경기도 연천군
2,춘장대솔내음캠핑장,춘장대솔내음캠핑장,야영장,36.166728,126.527285,충청남도 서천군 서면 춘장대로 151번길 20-6,NaN,000-0000-0000,50,14972.0,...,전기시설+상하수도시설+취사장+화장실+샤워장,소화기+대피소,NaN,NaN,30000,041-950-4018,충청남도 서천군청,2019-09-30,4580000,충청남도 서천군


In [238]:
def post_count(num, df, name, tag_nm):
    rows_list = []
    tag_list = df[tag_nm].values.tolist()
    
    today = datetime.datetime.today() 
    today_ymd = today.strftime('%Y%m%d')

    path = '../file/' 
    new_path = ''.join([path, today_ymd,'/'])
    file_nm = '_'.join(['camping', today_ymd])
    
    output_nm = ''.join([path, file_nm, ".csv"])
    mergeoutput_nm = ''.join([new_path, file_nm, ".csv"])
    
    cols = ['no','camp_nm','hashtag','num_posts','date_time']
    df_out = pd.DataFrame(columns=cols)

    try:
        os.mkdir(new_path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    chrome_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chrome_path, options=options)

    for i in range(num,len(tag_list)):
        driver.get('https://www.instagram.com/explore/tags/'+tag_list[i]+'/?hl=en') 
        time.sleep(2) 
        source = driver.page_source 
        data = bs(source, 'html.parser') 
        tag = data.find("span",{"class": "g47SY "}) 
        try: 
            val = tag.text 
            val = int(val.replace(',' , '')) 
        except AttributeError: 
            val = 0

        info = [i, df[name][i], tag_list[i], val, today]
        dict1 = dict(zip(cols,info))
        df_out = df_out.append(dict1, ignore_index=True)
        
        time.sleep(0.2)

        if i>0 and i%50 == 0:
            print(df_out.tail(5))
            driver.quit()
            df_out.to_csv(new_path+file_nm+"_"+f"{i:0>5}"+'.csv', index=False, header=False)
            
            df_out = pd.DataFrame(columns=cols)
            
            time.sleep(7)
            
            driver = webdriver.Chrome(chrome_path, options=options)

    df_out.to_csv(new_path+file_nm+"_"+f"{i:0>5}"+'.csv', index=False, header=False)
    driver.quit()
    
    with open(output_nm, 'wb') as outfile:
        for filename in glob.glob(new_path+file_nm+'*.csv'):
    #         print(filename)
            if filename == output_nm:
                # don't want to copy the output into the output
                continue

            with open(filename, 'rb') as readfile:
                readfile.readline()
                shutil.copyfileobj(readfile, outfile)

    outfile = pd.read_csv(output_nm, names=cols, index_col='no')
    print(outfile.head(10))
    outfile.to_csv(output_nm)

In [234]:
post_count(0, camping, 'name','name_modify')

Creation of the directory ../file/ failed
    no    camp_nm    hashtag num_posts                  date_time
46  46  청계동솔바람야영장  청계동솔바람야영장       143 2020-06-08 14:50:46.434935
47  47   도림사오토캠핑장   도림사오토캠핑장       466 2020-06-08 14:50:46.434935
48  48   곡성 키즈글램핑    곡성키즈글램핑       285 2020-06-08 14:50:46.434935
49  49  청계울낚시 캠핑장   청계울낚시캠핑장         0 2020-06-08 14:50:46.434935
50  50  자연애 품 캠핑장    자연애품캠핑장       172 2020-06-08 14:50:46.434935
     no      camp_nm      hashtag num_posts                  date_time
45   96   함평군 돌머리캠핑장    함평군돌머리캠핑장         0 2020-06-08 14:50:46.434935
46   97    석양마을주포캠핑장    석양마을주포캠핑장        23 2020-06-08 14:50:46.434935
47   98  함평자연생태공원캠핑장  함평자연생태공원캠핑장         0 2020-06-08 14:50:46.434935
48   99  함평나비마을서울캠핑장  함평나비마을서울캠핑장        24 2020-06-08 14:50:46.434935
49  100       수덕 캠핑장        수덕캠핑장       200 2020-06-08 14:50:46.434935
     no    camp_nm    hashtag num_posts                  date_time
45  146      솔밭야영장      솔밭야영장       217 2020-06-08 14:50:46.434935
46

In [237]:
with open(output_nm, 'wb') as outfile:
    for filename in glob.glob(new_path+file_nm+'*.csv'):
#         print(filename)
        if filename == output_nm:
            # don't want to copy the output into the output
            continue

        with open(filename, 'rb') as readfile:
            readfile.readline()
            shutil.copyfileobj(readfile, outfile)

outfile = pd.read_csv(output_nm, names=cols, index_col='no')

outfile.to_csv(output_nm)

        camp_nm      hashtag  num_posts                   date_time
no                                                                 
1   한탄강관광지오토캠핑장  한탄강관광지오토캠핑장       2667  2020-06-08 14:50:46.434935
2     춘장대솔내음캠핑장    춘장대솔내음캠핑장        243  2020-06-08 14:50:46.434935
3        늘푸른캠핑장       늘푸른캠핑장        564  2020-06-08 14:50:46.434935
4    서천갯벌체험글램핑장   서천갯벌체험글램핑장          4  2020-06-08 14:50:46.434935
5         백일야영장        백일야영장          1  2020-06-08 14:50:46.434935
6     춘장대나드리캠핑장    춘장대나드리캠핑장         46  2020-06-08 14:50:46.434935
7        춘장대캠핑장       춘장대캠핑장        274  2020-06-08 14:50:46.434935
8      춘장대오토캠핑장     춘장대오토캠핑장        152  2020-06-08 14:50:46.434935
9      서천해양글램핑장     서천해양글램핑장        140  2020-06-08 14:50:46.434935
10  서천금빛노을서울캠핑장  서천금빛노을서울캠핑장         11  2020-06-08 14:50:46.434935
